In [ ]:
from PIL import Image, ImageDraw, ImageFont
import geopandas as gpd
import datetime

effis = gpd.read_file('BRUTOS/EFFIS/modis.ba.poly.shp')
effis = effis[effis.COUNTRY == 'ES']
effis['year'] = effis.apply(lambda x: x.FIREDATE.split('-')[0], axis=1)
effis = effis[effis.year == '2022']
effis['SUP'] = effis.apply(lambda x: int(x.AREA_HA), axis=1)
gif = effis[effis.SUP > 500]
gif['fecha'] = gif.apply(lambda x: x.FIREDATE.split(' ')[0], axis=1)
gif['f'] = gif.apply(lambda x: datetime.datetime.strptime(x.fecha, '%Y-%m-%d'), axis=1)
gif['fecha_fin'] = gif.apply(lambda x: x.LASTUPDATE.split(' ')[0], axis=1)
gif['fecha_fin'] = gif.apply(lambda x: datetime.datetime.strptime(x.fecha_fin, '%Y-%m-%d'), axis=1)
display(gif)

In [ ]:
inicio = datetime.datetime.strptime('2022-06-01', '%Y-%m-%d')
meses = ['enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio', 'julio', 'agosto', 'septiembre', 'octubre', 'noviembre', 'diciembre']

for i in range(152):
    for k in ['00', '06', '12', '18']:
        fecha = inicio + datetime.timedelta(days=i)
        dia = int(fecha.strftime('%d'))
        mes = int(fecha.strftime('%m'))
        path = 'FINALES/isohipsas_850_{}_{}.png'.format(datetime.datetime.strftime(fecha, '%Y%m%d'), k)
        path_mapa = 'MAPAS/mapa_{}.png'.format(datetime.datetime.strftime(fecha, '%Y%m%d'))
        img = Image.new('RGB', (1920, 1080), color = 'white')
        wetter = Image.open(path)
        mapa = Image.open(path_mapa)

        fnt = ImageFont.truetype('/usr/share/fonts/truetype/freefont/FreeSerif.ttf', size=60)
        fnt2 = ImageFont.truetype('/usr/share/fonts/truetype/freefont/FreeSerif.ttf', size=26)
        fnt3 = ImageFont.truetype('/usr/share/fonts/truetype/freefont/FreeSerif.ttf', size=35)
        d = ImageDraw.Draw(img)
        d.text((350,50), "{} de {}".format(dia, meses[mes - 1]), font=fnt, fill=(0,0,0))
        d.text((1250,10), "GIFs activos", font=fnt, fill=(0,0,0))
        d.text((100,960), "Número GIFs temporada", font=fnt3, fill=(0,0,0))
        d.text((100,1000), "GIFs activos", font=fnt3, fill=(0,0,0))
        img.paste(wetter, (30, 150))
        img.paste(mapa, (1170, 500))

        a = gif[(gif.f <= fecha) & (gif.fecha_fin >= fecha)].sort_values(['f', 'COMMUNE'])
        incendios_temporada = gif[(gif.f >= inicio) & (gif.f <= fecha)]
        if len(a) > 0:
            contador = 0
            for j, incendios in a.iterrows():
                offset = 80 + 32 * contador
                if int(incendios.SUP) > 1000:
                    color = 'red'
                else:
                    color = 'black'
                d.text((1050, offset), "{} ({}) - {} ha".format(incendios.COMMUNE, incendios.PROVINCE, incendios.AREA_HA), fill=color, font=fnt2)
                contador += 1
        d.text((600,960), f"{len(incendios_temporada)}", font=fnt3, fill=(0,0,0))
        d.text((600,1000), f"{len(a)}", font=fnt3, fill=(0,0,0))
        img.save('FINALES/prueba_{}_{}.png'.format(datetime.datetime.strftime(fecha, '%Y%m%d'), k))
        img.close()